In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/face-mask-detection/dataset/without_mask/image_1.png
/kaggle/input/face-mask-detection/dataset/without_mask/image_213.png
/kaggle/input/face-mask-detection/dataset/without_mask/image_6.png
/kaggle/input/face-mask-detection/dataset/without_mask/image_37.png
/kaggle/input/face-mask-detection/dataset/without_mask/image_133.png
/kaggle/input/face-mask-detection/dataset/without_mask/image_22.png
/kaggle/input/face-mask-detection/dataset/without_mask/image_85.png
/kaggle/input/face-mask-detection/dataset/without_mask/image_150.png
/kaggle/input/face-mask-detection/dataset/without_mask/image_180.png
/kaggle/input/face-mask-detection/dataset/without_mask/image_138.png
/kaggle/input/face-mask-detection/dataset/without_mask/image_159.png
/kaggle/input/face-mask-detection/dataset/without_mask/image_93.png
/kaggle/input/face-mask-detection/dataset/without_mask/image_198.png
/kaggle/input/face-mask-detection/dataset/without_mask/image_156.png
/kaggle/input/face-mask-detection/dataset/

In [35]:
import torch
import numpy as np 
import matplotlib.pyplot as plt

from torch import nn
from torch.utils.data import Dataset
from torchvision import datasets, models, transforms
from torchvision.transforms import ToTensor
import torch.optim as optim
from torch.optim import lr_scheduler
from sklearn.model_selection import train_test_split
import time
import os
import copy

In [36]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"currently available device: {device}")

currently available device: cpu


In [37]:
data_dir = '/kaggle/input/face-mask-detection'
transform = transforms.Compose([
    #transforms.CenterCrop(10),
    #transforms.Grayscale(1),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(60),
    transforms.Resize((480,480)),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
])
dataset = datasets.ImageFolder(data_dir + '/dataset', 
                                  transform = transform)
train_set, val_set = train_idx, val_idx = train_test_split(list(range(len(dataset))), test_size=0.25)
dataset_sizes = {'train':len(train_set),'val':len(val_set) }
# class_names = train_data.classes


In [38]:
datasets = {}
datasets['train'] = torch.utils.data.Subset(dataset, train_set)
datasets['val'] = torch.utils.data.Subset(dataset, val_set)

In [39]:
dataloaders = {x:torch.utils.data.DataLoader(datasets[x],32, shuffle=True, num_workers=4) for x in ['train','val']}

In [40]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    print(f'Best val Acc: {best_acc:4f}')

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [53]:
class Emotion_classifier(nn.Module):
    def __init__(self):
        super(Emotion_classifier,self).__init__()
        
        self.conv1 = nn.Sequential(
        nn.Conv2d(3,32,9,stride=3),
        nn.ReLU(),
        nn.BatchNorm2d(32),
        nn.MaxPool2d(5),
        )
        
        self.conv2 = nn.Sequential(
        nn.Conv2d(32,64,9,),
        nn.ReLU(),
        nn.BatchNorm2d(64),
        nn.MaxPool2d(5),
        nn.Dropout(0.25),
        )
        
        
        self.flat = nn.Flatten()
        
        self.FC = nn.Sequential(
        nn.Linear(1024,256),
        nn.ReLU(),
        
        nn.Linear(256,2)
        )
    
    def forward(self,x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.flat(x)
        y = self.FC(x)
        
        return y

In [54]:
model_ft = Emotion_classifier().to(device)

criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer_ft = optim.Adam(model_ft.parameters(), lr=0.001)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=10, gamma=0.2)

In [55]:
model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,
                       num_epochs=50)

Epoch 0/49
----------
train Loss: 0.7916 Acc: 0.7182
val Loss: 1.4772 Acc: 0.5455

Epoch 1/49
----------
train Loss: 0.2600 Acc: 0.9091
val Loss: 0.3016 Acc: 0.8909

Epoch 2/49
----------
train Loss: 0.2351 Acc: 0.9242
val Loss: 0.2639 Acc: 0.9182

Epoch 3/49
----------
train Loss: 0.2144 Acc: 0.9303
val Loss: 0.3790 Acc: 0.9000

Epoch 4/49
----------
train Loss: 0.1760 Acc: 0.9303
val Loss: 0.2406 Acc: 0.9364

Epoch 5/49
----------
train Loss: 0.1925 Acc: 0.9182
val Loss: 0.2563 Acc: 0.9273

Epoch 6/49
----------
train Loss: 0.1459 Acc: 0.9455
val Loss: 0.2545 Acc: 0.9545

Epoch 7/49
----------
train Loss: 0.1475 Acc: 0.9455
val Loss: 0.2200 Acc: 0.9364

Epoch 8/49
----------
train Loss: 0.1424 Acc: 0.9485
val Loss: 0.2304 Acc: 0.9636

Epoch 9/49
----------
train Loss: 0.1532 Acc: 0.9424
val Loss: 0.1513 Acc: 0.9182

Epoch 10/49
----------
train Loss: 0.1023 Acc: 0.9667
val Loss: 0.2679 Acc: 0.9364

Epoch 11/49
----------
train Loss: 0.1112 Acc: 0.9697
val Loss: 0.2406 Acc: 0.9364

Ep

In [64]:
torch.onnx.export(model_ft,               # model being run
                  x,                         # model input (or a tuple for multiple inputs)
                  "/kaggle/working/Face_Mask_Classification.onnx",   # where to save the model (can be a file or file-like object)
                  export_params=True,        # store the trained parameter weights inside the model file
                  opset_version=10,          # the ONNX version to export the model to
                  do_constant_folding=True,  # whether to execute constant folding for optimization
                  input_names = ['input'],   # the model's input names
                  output_names = ['output'], # the model's output names
                  dynamic_axes={'input' : {0 : 'batch_size'},    # variable length axes
                                'output' : {0 : 'batch_size'}})

In [56]:
x = torch.randn(1, 3, 480, 480, requires_grad=True)
torch_out = model_ft(x)

In [57]:
torch_out


tensor([[ 0.7959, -1.4330]], grad_fn=<AddmmBackward0>)

In [59]:
import onnx

onnx_model = onnx.load("Face_Mask_Classification.onnx")
onnx.checker.check_model(onnx_model)

In [61]:
!pip install onnxruntime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 46.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 7.6 MB/s eta 0:00:00


In [62]:
import onnxruntime

ort_session = onnxruntime.InferenceSession("Face_Mask_Classification.onnx")

def to_numpy(tensor):
    return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()

# compute ONNX Runtime output prediction
ort_inputs = {ort_session.get_inputs()[0].name: to_numpy(x)}
ort_outs = ort_session.run(None, ort_inputs)

# compare ONNX Runtime and PyTorch results
np.testing.assert_allclose(to_numpy(torch_out), ort_outs[0], rtol=1e-03, atol=1e-05)

print("Exported model has been tested with ONNXRuntime, and the result looks good!")

Exported model has been tested with ONNXRuntime, and the result looks good!


In [63]:
ort_outs

[array([[ 0.79589504, -1.4329798 ]], dtype=float32)]